# Daten aus Excel einlesen

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt

pd.set_option('display.max_columns', 50)
pd.set_option('display.max_rows', 100)

Wenn man Daten anfragt, bekommt man neben .csv auch oft Excel-Files geschickt.
Das einlesen der Daten ist ähnlich wie bei .csvs. 

Zwei Dinge sind zu beachten:
1. Für die Arbeit mit Excel-daten benötigt pandas zusätzlich das Paket 'xlrd. Das muss ggf. noch installiert werden. (Hier ist das schon passiert.)
2. Excel-Files enthalten meist mehrere Tabellenblätter. Beim einlesen musst du angeben, welches Blatt (sheet_name) eingeladen werden soll. 

In [ ]:
df_2015 = pd.read_excel("./data/KHV_2015.xlsx", sheet_name = "KHV_2015", skiprows= 4)

In [ ]:
df_2015.sample()

In [ ]:
df_2015.columns

In [ ]:
df_2015 = df_2015[['Land', 'Kreis', 'Gemeinde', 'Name1', 'Ort', 'Traeger', 'T_Name1', 'Art', 'Betten_Ins']]

In [ ]:
df_2015['Jahr'] = '2015'

In [ ]:
df_2015.sample(3)

### Übung

### Welche journalistischen Fragen habt ihr an den Datensatz?

Manchmal hilft es, einen Datensatz zu befragen, wie man eine Interview-Partner*in befragt

Welche Fragen habt ihr an den Krankenhäuser-Datensatz? 

#### Fragen gesamt und NRW:

- wieviele Kliniken je Träger gibt es?
- wer sind die größten Klinikbetreiber?
- wieviele Unikliniken sind Privat?
- wieviele Betten privat und öffentlich?

#### Ländervergleich:

- meiste private Träger? 
- Karte: Cluster mit privaten?
- Betten/Einwohner
- Ost/West Vergleich

## Richtig spannend ist der Zeitverlauf

Um einen großen Datensatz zu erzeugen, der Veränderungen über die Jahre preisgibt, müssen wir mehrere Jahre einlesen. 
Problem oft: Je älter die Daten werden, desto größer die Wahrscheinlichkeit, dass sich die Datensätze über die Jahre verändert haben.


Wenn alle .xls-Dateien gleich aufgebaut wären, könnten wir eine Funktion schreiben, die alle Datensätze einliest und miteinander verknüpft.

Schau dir unter ./data mal die 'KHV_' - Dateien an:

- alle beginnen mit 'KHV_', gefolgt vom Jahr
- es gibt drei verschiedene Formate .xls, .XLS, .xlsx
- 1992 heißt das Datenblatt nach dem wir suchen 'KHV'
- 1995 heißt es KH
- 2003 bis 2005 gibt es ein Datenblatt 'KHV', immer als 5. Blatt im Dokument
- 2015 heißt das Datenblatt 'KHV_2015' und ist ebenfalls das 5. Blatt im Dokument

### Die Rohdaten sind lava!
Die Versuchung ist groß, mal eben die Ungleichheiten händisch in den Excel-Files zu bereinigen.
Diese Änderungen würden aber unsere Datenquelle verändern und wären nicht dokumentiert, also schwer wiederholbar für die Abnahme oder auch für die Leser*innen.

Entsprechend wählen wir einen halbautomatisierten, dokumentierten Weg im Jupyter Notebook:

### Nice to know

Um auf das Dinge im Betriebssystem zuzugreifen, gibt es das Python Modul `os`

In [ ]:
import os

In [ ]:
# Alle Dateien anzeigen, die im Ordner 'data' liegen
path = './data'
files = os.listdir(path)
files

Um Dateien nach Mustern zu durchsuchen, gibt es das Python Modul `glob`

In [ ]:
from glob import glob

In [ ]:
# Die Dateien in einer Liste speichern, die mit 'KHV_' anfangen, das Sternchen ersetzt beliebige Zeichen
files_xls = glob('./data/KHV_*')
files_xls.sort()
files_xls

 Mithilfe dieser Dateiliste könnten wir jetzt unsere Funktion schreiben, die jede Datei durchläuft und die entsprechend bereinigten dataframes anlegt. Da die Dateien aber kleine Unterschiede aufweisen, gehen wir jahrweise vor:

#### 1992

In [ ]:
df_1992 = pd.read_excel('./data/KHV_1992.xls', dtype='object')

In [ ]:
df_1992.sample()

In [ ]:
df_1992.columns

In [ ]:
# Spalten so umbenennen, das sie dem Standard der späteren Jahre entsprechen
df_1992 = df_1992.rename(columns={'LAND': 'Land', 'KREIS': 'Kreis', 'GEMEINDE': 'Gemeinde', 'NAME1': 'Name1', 'ORT': 'Ort', 'TRAEGER': 'Traeger', 'NAMET1': 'T_Name1', 'ART':'Art', 'BETTEN_I': 'Betten_Ins'})

In [ ]:
# Benötigte Spalten auswählen
df_1992 = df_1992[['Land', 'Kreis', 'Gemeinde', 'Name1', 'Ort', 'Traeger', 'T_Name1', 'Art', 'Betten_Ins']]

Es fehlt noch eine Spalte 'Jahr', in der das jeweilige Jahr aus dem Dateinamen eingetragen wird.

### Muster erkennen mit regulären Ausdrücken

Die Jahre können wir automatisch aus den Zahlen in den Dateinamen auslesen. 

Das Modul `re` ermöglicht nach regulären Ausdrücken zu suchen: Zum Beispiel alle Zahlen in einem String zu finden. 

In [ ]:
import re

Beispiele für reguläre Ausdrücke (RegEx):

**Regular Expression Character Classes**

| Expression | Explanation |
| --- | --- |
| [ab-d] | One character of: a, b, c, d |
| [^ab-d] | One character except: a, b, c, d |
| [\b] | Backspace character |
| \d | One digit |
| \D | One non-digit |
| \s | One whitespace |
| \S | One non-whitespace |
| \w | One word character |
| \W | One non-word character |

**Regular Expression Quantifiers**

| Expression | Explanation |
| --- | --- |
| \* | 0 or more |
| \+ | 1 or more |
| ? | 0 or 1 |
| {2} | Exactly 2 |
| {2, 5} | Between 2 and 5 |
| {2,} | 2 or more |
| (,5} | Up to 5 |

In [ ]:
teststring = 'KHV_1992.xls'

In [ ]:
# Mit (\d) findest Du Zahlen in deinem Teststring, das + gibt an, dass wir mehr als eine Zahl suchen.
re.search(r'\d+', teststring).group()

In [ ]:
# Damit wir nochher mehrere Excel-Dateien auf einmal verarbeiten können, schreiben wir eine Funktion, die jeweils das Jahr zurückgibt:
def get_numbers_from_filename(filename):
    return re.search(r'\d+', filename).group()

In [ ]:
# Funktion testweise ausführen
get_numbers_from_filename("KHV_1992.xls")

In [ ]:
# Spalte 'Jahr' setzen
df_1992["Jahr"] = get_numbers_from_filename("KHV_1992.xls")

In [ ]:
df_1992.sample()

### 1995

Für 1995 müssen wir ähnliche, aber leider nicht exakt dieselben Schritte ausführen:

In [ ]:
# Einlesen, diesmal mit sheet_name, da das Blatt, welches wir haben wollen, nicht das erste ist. 
df_1995 = pd.read_excel('./data/KHV_1995.XLS', sheet_name='KH', dtype='object')

In [ ]:
# Spalten so umbenennen, das sie dem Standard der späteren Jahre entsprechen
df_1995 = df_1995.rename(columns={'LAND': 'Land', 'KREIS': 'Kreis', 'GEMEINDE': 'Gemeinde', 'NAME1': 'Name1', 'ORT': 'Ort', 'TRAEGER': 'Traeger', 'NAMET1': 'T_Name1', 'ART':'Art', 'BETTEN_I': 'Betten_Ins'})

In [ ]:
# Benötigte Spalten auswählen
df_1995 = df_1995[['Land', 'Kreis', 'Gemeinde', 'Name1', 'Ort', 'Traeger', 'T_Name1', 'Art', 'Betten_Ins']]

In [ ]:
# Jahr
df_1995["Jahr"] = get_numbers_from_filename("KHV_1995.XLS")

In [ ]:
df_1995.sample()

### 2003 - 2005

2003 bis 2005 können wir in einem einlesen 

In [ ]:
filenames = [
 './data/KHV_2003.xls',
 './data/KHV_2004.xls',
 './data/KHV_2005.xls',]

In [ ]:
df_from_each_file = []

for file in filenames:
    data = pd.read_excel(file, sheet_name=4, dtype='object')
    data["Jahr"] = get_numbers_from_filename(file)
    data = data[['Land', 'Kreis', 'Gemeinde', 'Name1', 'Ort', 'Traeger', 'T_Name1', 'Art', 'Betten_Ins', 'Jahr']]
    df_from_each_file.append(data)

Hast Du gemerkt, dass wir diesmal 'sheet_name=4' gesetzt haben?
Das suchst das 5. Datenblatt heraus, denn Python fängt mit dem Zählen bei 0 an.

Um die DataFrames zu verknüpfen nutzen wir `pd.concat`. Die Funktion nimmt eine Liste von Dataframes und klebt sie untereinander. Das funktioniert aber nur dann gut, wenn die Spaltennamen übereinstimmen.

In [ ]:
df_2003_2005 = pd.concat(df_from_each_file, ignore_index=True)

In [ ]:
df_2003_2005.sample()

In [ ]:
# Immerhin 6369 Zeilen in einem eingelesen 
len(df_2003_2005)

### Jetzt bringen wir alles zusammen

In [ ]:
dfs = [df_1992, df_1995, df_2003_2005, df_2015]

In [ ]:
df = pd.concat(dfs, ignore_index=True)

In [ ]:
df.shape

In [ ]:
df.sample()

## Entwicklung der privaten Träger

In den Excel-Dateien findet sich folgende Erklärung zu den Trägern:

Art des Trägers und Rechtsform des Krankenhauses

1 = öffentlich in öffentlich-rechtlicher Form, rechtlich unselbstständig

2 = öffentlich in öffentlich-rechtlicher Form, rechtlich selbstständig

3 = öffentlich in privatrechtlicher Form

4 = freigemeinnützig

5 = privat


In [ ]:
# Wie oft kommt der Wert 5 in der Spalte Träger vor?
df.Traeger.value_counts()
# Typ 5 kommt doppelt vor, das liegt daran, dass die Werte einmal als int und einmal als str vorhanden sind.

In [ ]:
# Datentypen checken
df.dtypes

In [ ]:
# Datentyp für die Spalte Träger zu int ändern
df.Traeger = df.Traeger.astype('int')

In [ ]:
# Geklappt?
df.dtypes

In [ ]:
# Im Datensatz sind private Krankenhäuser mit Träger '5' angegeben. Darauf filtern wir.
df = df[df.Traeger == 5]

In [ ]:
df.shape

In [ ]:
private = df.groupby(['Jahr']).count()
private

Im Jahr 1992 und 1995 gab es noch keine privaten Krankenhäuser.

In [ ]:
private.plot(y='Land', kind="bar", figsize=(15,6),  legend=False, 
                                           title="Entwicklung private Krankenhäuser",)

Geschafft!

Bitte lies noch das

**[ >> Api Grundlagen Notebook](07_api.ipynb)**